In [ ]:
import numpy as np
import sympy as sp
from sympy import *
from sympy.matrices import Matrix
from sympy.abc import z,t
from math import comb
import math


In [ ]:
def rational_from_contfrac(cf):
  # retrieves rational form [p,q] from continuous fraction cf
  n = len(cf) - 1
  total = [cf[n], 1]
  while n > 0:
    n = n-1
    total = [total[1], total[0]]
    total = [total[0] + cf[n] * total[1], total[1]]

  d = math.gcd(total[0], total[1])
  return [total[0]/d, total[1]/d]


def v3_from_evencontfrac(cf):
  # calculates v3 from even continuous fraction cf, cf even length each index even
  v3 = 0
  cf = [x/2 for x in cf]
  cf_even = cf[1::2]
  cf_odd = cf[0::2]
  for i in range(len(cf_even)):
    v3 = v3 + (cf_even[i]) * ((sum(cf_odd[0:i+1]))**2)

  for i in range(len(cf_odd)):
    v3 = v3 - (cf_odd[i]) * ((sum(cf_even[i:]))**2)

  return 0.5*v3


def genus(cf_even):
    return len(cf_even) / 2


def even_cf(n, d):
  # positive continuous fraction
  if d == 0:
    return []
  q = int(n//d)
  if q%2 != 0:
    q = q + 1
  r = n-q*d
  return [q] + even_cf(d , r)


def good_frac(p,q):
  if q % 2 == 0:
    return p,q
  else:
    if abs(q+p) < p:
      return p, q+p
    else:
      return p, q-p

def det(conway):
  return abs(conway.subs(z**2, -4))


#find jones polynomial
def jones_poly(cf, t):
  if (len(cf) == 0):
    return 1
  if (len(cf) == 1):
    if (cf[0] == 1):
      return 1
    if (cf[0] == 0):
      return (-t - t**(-1))
  if (cf[0] == 0):
    return jones_poly(cf[2::], t)
  elif (cf[0] == 1):
    new_cf = cf[1::]
    new_cf[0] = new_cf[0] + 1
    return jones_poly(new_cf, t)
  if (len(cf) % 2 == 1):
    new_cf1 = cf[:]
    new_cf1[0] = new_cf1[0] - 1
    new_cf2 = cf[:]
    new_cf2[0] = new_cf2[0] - 2
    return sp.expand((t**4) * jones_poly(new_cf2, t) + (t**3 - t) * jones_poly(new_cf1, t))
  else:
    new_cf1 = cf[1::]
    new_cf2 = cf[:]
    new_cf2[0] = new_cf2[0] - 2
    return sp.expand((t**4) * jones_poly(new_cf2, t) + (t**3 - t) * jones_poly(new_cf1, t))

#find conway polynomial with recursion
def conway_poly_recur(cf, z):
  if (len(cf) == 0):
    return 1
  if (len(cf) == 1):
    if (cf[0] == 1):
      return 1
    if (cf[0] == 0):
      return 0
  if (cf[0] == 0):
    return conway_poly_recur(cf[2::], z)
  elif (cf[0] == 1):
    new_cf = cf[1::]
    new_cf[0] = new_cf[0] + 1
    return conway_poly_recur(new_cf, z)
  if (len(cf) % 2 == 1):
    new_cf1 = cf[:]
    new_cf1[0] = new_cf1[0] - 1
    new_cf2 = cf[:]
    new_cf2[0] = new_cf2[0] - 2
    return sp.expand(conway_poly_recur(new_cf2, z) + z * conway_poly_recur(new_cf1, z))
  else:
    new_cf1 = cf[1::]
    new_cf2 = cf[:]
    new_cf2[0] = new_cf2[0] - 2
    return sp.expand(conway_poly_recur(new_cf2, z) + z * conway_poly_recur(new_cf1, z))

#find conway polynomial with recursion and database
def new_conway_poly_recur(cf, z, database):
    ratio = tuple(rational_from_contfrac(cf))
    if (ratio in database):
      return database[ratio]
    if (len(cf) == 0):
      return 1
    if (len(cf) == 1):
      if (cf[0] == 1):
        return 1
      if (cf[0] == 0):
        return 0
    if (cf[0] == 0):
      new_poly = new_conway_poly_recur(cf[2::], z, database)
    elif (cf[0] == 1):
      new_cf = cf[1::]
      new_cf[0] = new_cf[0] + 1
      new_poly = new_conway_poly_recur(new_cf, z, database)
    elif (len(cf) % 2 == 1):
      new_cf1 = cf[:]
      new_cf1[0] = new_cf1[0] - 1
      new_cf2 = cf[:]
      new_cf2[0] = new_cf2[0] - 2
      new_poly = sp.expand(new_conway_poly_recur(new_cf2, z, database) + z * new_conway_poly_recur(new_cf1, z, database))
    else:
      new_cf1 = cf[1::]
      new_cf2 = cf[:]
      new_cf2[0] = new_cf2[0] - 2
      new_poly = sp.expand(new_conway_poly_recur(new_cf2, z, database) + z * new_conway_poly_recur(new_cf1, z, database))
    database[ratio] = new_poly
    return new_poly

def count_knots(max_total):
  result = 0
  for j in range(3,max_total,2):
    for i in range(1,int((j-1)*2/3)+2,2):
      result += len(findNDigitNums(i, j))
  return(result)

def cosmetic_surg_check(p,q, v3, det, a2, genus, a4, i):
  #returns true if Konstantinos' equation is satisfied
  #return [str(int(p))+"/"+str(int(q)), int(v3*((det/2 + genus - 3/2)+2*genus - 1)), 7*(a2**2) - a2 - 10*a4 , v3*((0.5*det + genus - 3/2)+2*genus - 1)== 7*(a2**2) - a2 - 10*a4, i]
  return v3*((0.5*det + genus - 3/2)+2*genus - 1)== 7*(a2**2) - a2 - 10*a4

def findNDigitNumsUtil(n, total, out, index, results):
    # If number becomes N-digit
    if (index == n - 1):
      out[index] = total
      results += [out[:]]
      return

    if (index % 2 == 0):
      min_total = 0
      for i in range(n - index - 1):
        if (i % 2 == 0):
          min_total += 2
        else:
          min_total += 1
      for j in range(1, total - min_total + 1):
        out[index] = j
        findNDigitNumsUtil(n, total - j, out, index + 1, results)
    else:
      min_total = 0
      for i in range(n - index - 1):
        if (i % 2 == 0):
          min_total += 1
        else:
          min_total += 2
      for j in range(2, total - min_total + 1, 2):
        out[index] = j
        findNDigitNumsUtil(n, total - j, out, index + 1, results)

# This is mainly a wrapper over findNDigitNumsUtil.
# It explicitly handles leading digit
def findNDigitNums( n, total):

    # output array to store N-digit numbers
    out = [0] * (n)
    results = []

    findNDigitNumsUtil(n, total, out, 0, results)
    return results


def check_conj_recur(cf_odd, database, i):
  [p,q] = rational_from_contfrac(cf_odd)
  #if not (pos(p,q)):
  #  return "Not a Positive 2-bridge Knot"
  gp, gq = good_frac(p,q)
  cf_even = even_cf(gp,gq)
  v3 = int(v3_from_evencontfrac(cf_even))
  z = sp.symbols('z')
  conway_poly = new_conway_poly_recur(cf_odd, z, database)
  a2 = conway_poly.coeff(z, 2)
  a4 = conway_poly.coeff(z, 4)
  determinant = int(det(conway_poly))
  g = genus(cf_even)
  return cosmetic_surg_check(p,q, v3, determinant, a2, g, a4, i)



In [ ]:
def main(max_total):
  database = dict()
  i = 0
  for total in range(3,max_total+1,2):
    max_n = int(2*(total-1)/3)+1
    for cf_len in range(1, max_n+1, 2):
      arr = findNDigitNums(cf_len, total)
      for cf_odd in arr:
        if check_conj_recur(cf_odd, database, i):
          print([cf_odd, rational_from_contfrac(cf_odd)])
          return
        i+=1
  print('Conjecture Satisfied for Positive 2-Bridge Knots Up To ' + str(max_total) + ' Crossings')
  print(len(database))

In [ ]:
main(31)